In [ ]:
#HANA connector
import hana_ml.dataframe as dataframe
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
#Python stuff
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import tree
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from matplotlib import pyplot
import imblearn
import numpy as np

In [ ]:
conn = NotebookConnectionContext(connectionId = 'hanapoc')

Get data

In [ ]:
sql = (
"select "
"top 1000 DAYS_BETWEEN(ID_CREDAT, FIRST_ORDER) as TIMETOSHOP, *, "
"CASE WHEN CUST_CLASS = 'Z0' THEN '0' WHEN CUST_CLASS = 'Z1' then '1' end as CUST_CLASS_INDC, "
"CASE WHEN (KITCHENS_FIRST_N_WKS_I - KITCHENS_FIRST_N_WKS_R) > 0 THEN '1' ELSE '0' end as KITCHEN_INDC, "
"CASE WHEN (ORDERS_TOTAL_I - ORDERS_FIRST_N_WKS_I) > 0 THEN '1' else '0' end as REPEATCUSTOMER, "
"CASE WHEN ORDERS_FIRST_N_WKS = '1' THEN '1' else '0' end as SINGLEORDER_FIRST_N_WKS, "
"CASE WHEN ID_CREDAT = FIRST_ORDER THEN '1' else '0' end as INSTANTSHOP "
"from TBL_BWH_CUSTOMERSUMMARY_8WKS "
#"where NET_VALUE_TOTAL > '0' and ID_CREDAT > '2018' and ID_CREDAT < '2021-10-01' " 
"where ID_CREDAT > '2019' and ID_CREDAT < '2021' " 
"ORDER BY RAND() LIMIT 1000"
)
    
cust_sel = conn.sql(sql)
customers = cust_sel.collect()
customers

Show histogram of TIMETOSHOP with 100 bins...

In [ ]:
customers['TIMETOSHOP'].plot(kind='hist',bins=100,figsize=(15, 10))

Get means for repeat customers and all

In [ ]:
customers.loc[customers['REPEATCUSTOMER'] == '1']['TIMETOSHOP'].describe()

In [ ]:
customers['TIMETOSHOP'].describe()